### Azure ML Resources
- [Tune Hyperparameters](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-tune-hyperparameters)

In [6]:
import sys
from distutils import dir_util

import azureml.core
from azureml.core import Experiment
from azureml.core.workspace import Workspace
from azureml.core.compute import ComputeTarget
from azureml.train.estimator import Estimator
from azureml.train.hyperdrive import *
from azureml.widgets import RunDetails

ws = Workspace.from_config('/home/common/notebooks/AML/config.json')
sys.path.append('/home/zer0ne/src/RL')

Found the config file in: /home/common/notebooks/AML/config.json


In [2]:
p100_1_low = ComputeTarget(workspace=ws, name='p100-1-low')  # each node has 1 P100
p100_1_low2 = ComputeTarget(workspace=ws, name='p100-1-low2')  # each node has 1 P100
p100_1 = ComputeTarget(workspace=ws, name='p100-1')  # each node has 1 P100; don't use this
p100_2 = ComputeTarget(workspace=ws, name='p100-2')  # each node has 2 P100s
k80_low = ComputeTarget(workspace=ws, name='k80-low')  # each node has 1 K80

metric = "Average Return"

def train(name, cluster, args, hyper_params=None,
          source_directory='./MBRL', 
          entry_script='train.py', 
          docker_image='zer0n/rl',
          total_runs=100):
    trainer = Estimator(source_directory=source_directory,
                        script_params=args, 
                        compute_target=cluster,
                        entry_script=entry_script,
                        custom_docker_image=docker_image, 
                        use_gpu=True, 
                        user_managed=True)
    experiment = Experiment(ws, name=name)
    
    if hyper_params:
        sweeper = HyperDriveRunConfig(estimator=trainer,
                                      hyperparameter_sampling=RandomParameterSampling(hyper_params), 
                                      primary_metric_name=metric, 
                                      primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                      policy=MedianStoppingPolicy(evaluation_interval=1, delay_evaluation=5),
                                      max_total_runs=total_runs)
        return experiment.submit(sweeper)
    else:
        return experiment.submit(trainer)

def visualize(run):
    RunDetails(run).show()
    
def get_run(run_id: str):
    experiment = ws.experiments[run_id.split('_')[0]]
    return azureml.core.get_run(experiment, run_id)    

def copy_log_dir(run):
    from arguments import Arguments, parse_args
    args = Arguments()
    cmd_args = run.get_details()['runDefinition']['arguments']
    parse_args(args, cmd_args)
    log_dir = args.log_dir.replace('../', '/mnt/azureml/azureml/')
    dir_util.copy_tree(log_dir, f'/home/share/log/{cmd_args[1]}/')

In [9]:
pendulum = 'Pendulum-v0'
half_cheetah = 'HalfCheetah-v2'
humanoid = 'Humanoid-v0'